In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_sample_images

images=load_sample_images()['images']
print('Length of images:')
print(len(images))
print('Shape of images:')
print(np.shape(images))
#there are 2 images here, each image has dimensions 427x640, there are 3 color channels(rgb)

images=tf.keras.layers.CenterCrop(height=70,width=120)(images)
#retains only center patch of each image
images=tf.keras.layers.Rescaling(scale=1/255)(images)
#this rescales every pixel value to 0-1 range by dividing them by 255
#coloe and intensity of pixels remain same as relative diff between pixel values is retained
print('Shape of images after preprocessing:')
print(images.shape)

conv_layer=tf.keras.layers.Conv2D(filters=32,kernel_size=7) #32 different feature maps created
fmaps=conv_layer(images)
print('Shape of created feature maps before zero padding:')
print(fmaps.shape)
#width and height changes because we lose pixels due to lack of zero padding

conv_layer=tf.keras.layers.Conv2D(filters=32,kernel_size=7, padding='same') #32 different feature maps created
fmaps=conv_layer(images)
print('Shape of created feature maps after zero padding:')
print(fmaps.shape)

kernels,biases=conv_layer.get_weights()
print('Shape of kernels:')
print(kernels.shape)
#kernels are the filters of each convolutional layer, store the weights of the filters
#7,7 is size of kernel, 3 represents number of color channels, 32 represents number of filters
print('Shape of biases:')
print(biases.shape)
#every filter has a bias term

#POOLING LAYERS:
max_pool=tf.keras.layers.MaxPool2D(pool_size=2)
#you can also use AvgPool2D for average pooling layer
global_avg_pool=tf.keras.layers.GlobalAvgPool2D()
#this layer computes mean of entire feature map and outputs only 1 value per color channel(no receptive field logic)

global_res=global_avg_pool(images)
print('Output of global average 2D layer:')
print(global_res)#2 lists for 2 images


Length of images:
2
Shape of images:
(2, 427, 640, 3)
Shape of images after preprocessing:
(2, 70, 120, 3)
Shape of created feature maps before zero padding:
(2, 64, 114, 32)
Shape of created feature maps after zero padding:
(2, 70, 120, 32)
Shape of kernels:
(7, 7, 3, 32)
Shape of biases:
(32,)
Output of global average 2D layer:
tf.Tensor(
[[0.64338624 0.5971759  0.5824972 ]
 [0.76306933 0.26011038 0.10849128]], shape=(2, 3), dtype=float32)


In [2]:
from functools import partial
from tensorflow.keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

train_images,val_images,train_labels,val_labels=train_test_split(train_images,train_labels,test_size=0.2,random_state=42)
train_images=np.expand_dims(train_images,axis=-1)
test_images=np.expand_dims(test_images,axis=-1)
val_images=np.expand_dims(val_images,axis=-1)

#makes all images grayscale(1 color dimension)
DefaultConv2D= partial(tf.keras.layers.Conv2D, kernel_size=3,padding='same',activation='relu', kernel_initializer='he_normal')
model=tf.keras.Sequential([
    DefaultConv2D(filters=64,kernel_size=7,input_shape=[28,28,1]), #specifies dimensions of input data it is working with
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    tf.keras.layers.MaxPool2D(),#MaxPool2D works on every generated feature map
    tf.keras.layers.Flatten(),# all diff feature maps due to filters concatenated together
    tf.keras.layers.Dense(units=128,activation='relu',kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=64,activation='relu',kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10,activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
history=model.fit(train_images,train_labels,validation_data=[val_images,val_labels],epochs=10)
print('Evaluation of this model on test data:')
print(model.evaluate(test_images,test_labels))

Epoch 1/10
1500/1500 [==============================] - 321s 213ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1002
Epoch 2/10
1500/1500 [==============================] - 316s 211ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1002
Epoch 3/10
1500/1500 [==============================] - 309s 206ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1002
Epoch 4/10
1500/1500 [==============================] - 311s 207ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1002
Epoch 5/10
1500/1500 [==============================] - 313s 208ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1002
Epoch 6/10
1500/1500 [==============================] - 326s 217ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1002
Epoch 7/10
1500/1500 [==============================] - 327s 218ms/step - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.1002
Epoch 